In [ ]:
from pymongo import MongoClient
import json
import ipywidgets as wgt
from IPython.display import display
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import tweepy

api_key = "6gbFz4BxVfJo1gCOBWrUIMsWg" # <---- Add your API Key
api_secret = "rw9B4CSQpyNF13vgDNS9D7ahXohexkoyWnqOBKxqi55n7z6PCR" # <---- Add your API Secret
access_token = "764000256258211843-MyKIzwUTpKJVsWNObQesqmPeF5JcnGt" # <---- Add your access token
access_token_secret = "OiGqWVmXLPkChPk0A3S0QHGCPbviK793p8T30uxAWj6Q1" # <---- Add your access token secret

auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
# Connect to MongoDB locally with database named 'project-bigdata'
MONGO_HOST = 'mongodb://localhost/projectbigdata'

In [ ]:
class MyStreamListener(tweepy.StreamListener):
    '''
    Listener class with progress visualization
    '''
    
    counter = 0
    
    def __init__(self, max_tweets=10000, *args, **kwargs):
        # Initialize the class with max tweets accepted 1000
        self.max_tweets = max_tweets
        self.counter = 0
        super().__init__(*args, **kwargs)
    
    def on_connect(self):
        # Called initially to connect to the streaming API
        self.counter = 0
        self.start_time = datetime.now()
    
    def on_data(self, data):
        # Connects to MongoDB and store the tweets
        
        try:
            client = MongoClient(MONGO_HOST)
            
            # Use project-bigdata database
            db = client.projectbigdata
            
            # Decode the JSON from Twitter
            datajson = json.loads(data)
            
            # Insert the data into the MongoDB into a collection called blockchain-tweets
            db.bitcointweets.insert(datajson)
            
            # Increment counter everytime new data arrives
            self.counter += 1
            
            # Visualization of progress
            if self.counter % 1 == 0:
                value = int(100.00 * self.counter / self.max_tweets)
                mining_time = datetime.now() - self.start_time
                progress_bar.value = value
                html_value = """<span class="label label-primary">Tweets/Sec: %.1f</span>""" % (self.counter / max([1,mining_time.seconds]))
                html_value += """ <span class="label label-success">Progress: %.1f%%</span>""" % (self.counter / self.max_tweets * 100.0)
                html_value += """ <span class="label label-info">ETA: %.1f Sec</span>""" % ((self.max_tweets - self.counter) / (self.counter / max([1,mining_time.seconds])))
                wgt_status.value = html_value
                #print("%s/%s" % (self.counter, self.max_tweets))
                if self.counter >= self.max_tweets:
                    myStream.disconnect()
                    print("Finished")
                    print("Total Mining Time: %s" % (mining_time))
                    print("Tweets/Sec: %.1f" % (self.max_tweets / mining_time.seconds))
                    progress_bar.value = 0
        
        except Exception as e:
            print(e)

In [ ]:
# The actual listener 
myStreamListener = MyStreamListener(max_tweets=10000)
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [ ]:
col = MongoClient()["projectbigdata"]["bitcointweets"]
col.count()

In [ ]:
keywords = ["Bitcoin"]

# Visualize a progress bar to track progress
progress_bar = wgt.IntProgress(value=0)
display(progress_bar)
wgt_status = wgt.HTML(value="""<span class="label label-primary">Tweets/Sec: 0.0</span>""")
display(wgt_status)

# Start a filter with an error counter of 20
for error_counter in range(10):
    try:
        myStream.filter(track=keywords)
        print("Tweets collected: %s" % myStream.listener.counter)
        print("Total tweets in collection: %s" % col.count())
        break
    except:
        print("ERROR# %s" % (error_counter + 1))

In [ ]:
col.find_one()

In [ ]:
import pandas as pd 

#loading data ke data frame

data = pd.DataFrame()
data["text"] = list(map(lambda tweet: tweet['text'], tweets_data))
data['lang'] = list(map(lambda tweet: tweet['lang'], tweets_data))
data['created_at'] = list(map(lambda tweet: tweet['created_at'], tweets_data))
data['country'] = list(map(lambda tweet: tweet['place']['country'] if tweet['place'] != None else None, tweets_data))

In [ ]:
print ("Jumlah Data Tweets: ", len(data))

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data['text'][0]